In [1]:
import vk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime, date
from requests import ReadTimeout
from ipywidgets import IntProgress
from IPython.display import display

In [2]:
user_id = ''
app_id = ''
login = ''
password = ''
session = vk.AuthSession(app_id=app_id, user_login=login, user_password=password)
api = vk.API(session)

In [3]:
groups_df = pd.read_json('C:/Users/Taya/Desktop/Coursera analytics/Python/python-netology/finals/groups.json').sort_values(by=['FansNumber'], ascending=False).head(5)

In [4]:
groups_df[['GroupName','FansNumber']]

,GroupName,FansNumber
0,Esquire,56
1,KudaGo: Москва,51
2,Шедевры рекламы,36
3,Science|Наука,35
4,Лепра,34


In [5]:
def get_group_info_df(group_id):
    response = api.groups.getMembers(group_id=group_id, fields=['sex', 'bdate'])
    users_count = response['count']
    users_list = response['users']
    df = pd.DataFrame(users_list)
    progress = IntProgress(min=0, max=users_count, value=len(df.index))
    display(progress)
    while len(df.index) < users_count:
        try: 
            offset = len(df.index)
            df_to_add = pd.DataFrame(api.groups.getMembers(group_id=group_id, fields=['sex', 'bdate'], offset=offset)['users'])
            df = df.merge(df_to_add)
            progress.value = len(df.index)
        except:
            pass
    return df

In [8]:
def calculate_age(bdate_str):
    if bdate_str.count(".") != 2:
        return 0
    bdate = datetime.strptime(bdate_str, '%d.%m.%Y').date()
    today = date.today()
    return today.year - bdate.year - ((today.month, today.day) < (bdate.month, bdate.day))

In [9]:
def display_ages_chart(df, group_name):
    ages = df["bdate"].map(calculate_age)
    pd.cut(ages, [10, 15, 20, 30, 40, 80, 1000], labels=["10-15", "16-20","21-30", "31-40", "41-80", "80 и старше"]).value_counts().plot.pie(title="Возраст участников группы"+"group_name")
    
def display_sex_chart(df, group_name):
    df['sex'].replace(1,"Ж").replace(2,"М").value_counts().plot.bar(title="Распределение полов")

In [ ]:
for i, row in groups_df.iterrows():
    df = get_group_info_df(row["GroupID"])
    display_ages_chart(df, row["GroupName"])
    display_sex_chart(df, row["GroupName"])